In [5]:

# filename1= 'Triage-Counterfactual_CoT_gpt-4o_json0_99_20241125_173640.csv'
# filename2 = 'Triage-Counterfactual_CoT_gpt-4o_json100_1000_20241124_173026.csv'
# filename3 = 'Triage-Counterfactual_CoT_gpt-4o_json1000_3000_20241124_211048.csv'
# filepaths = [
#     '../results/Triage-Counterfactual/' + filename1,
#     '../results/Triage-Counterfactual/' + filename2,
#     '../results/Triage-Counterfactual/' + filename3
# ]

# # Read and combine all dataframes
# dfs = [pd.read_csv(filepath) for filepath in filepaths]
# dfs[2] = dfs[2].loc[1:len(dfs[2])-1]
# combined_df = pd.concat(dfs, ignore_index=True)

# # Save the combined dataframe to a new file if needed
# output_filepath = '../results/Triage-Counterfactual/Triage-Counterfactual_CoT_gpt-4o_json0_3000.csv'
# combined_df.to_csv(output_filepath, index=False)

### Create Embeddings for KATE Few Shot Prediction

In [1]:
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import numpy as np
from tqdm import tqdm

/opt/homebrew/anaconda3/envs/llm-medical-bias/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
import pandas as pd
train = pd.read_csv('../data/mimic-iv-private/anchor_year_group_datasets/2014_-_2016/small_train_dataset.csv')
test = pd.read_csv('../data/mimic-iv-private/anchor_year_group_datasets/2017_-_2019/test_dataset.csv')

In [35]:
train['chiefcomplaint'] = train['chiefcomplaint'].astype(str)
test['chiefcomplaint'] = test['chiefcomplaint'].astype(str)

# Load the pre-trained SentenceTransformer model for clinical text
# This model is fine-tuned for biomedical and clinical text embeddings
model_name = 'pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb'
model = SentenceTransformer(model_name)

# from transformers import AutoTokenizer, AutoModel
# tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
# model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")


In [20]:
# List of vital signs columns
vital_signs_cols = ['temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp', 'pain']

# Ensure vital signs are numeric and handle missing values
train[vital_signs_cols] = train[vital_signs_cols].apply(pd.to_numeric, errors='coerce')

# Normalize vital signs using Min-Max scaling
scaler = MinMaxScaler()
vital_signs_normalized = scaler.fit_transform(train[vital_signs_cols])

In [21]:
print(f"NaN in embeddings: {np.isnan(vital_signs_normalized).any()}")
train[vital_signs_cols] = train[vital_signs_cols].fillna(train[vital_signs_cols].mean())
vital_signs_normalized = scaler.fit_transform(train[vital_signs_cols])
print(f"NaN in embeddings: {np.isnan(vital_signs_normalized).any()}")
train[vital_signs_cols] = vital_signs_normalized


NaN in embeddings: True
NaN in embeddings: False


In [24]:
categorical_cols = ['gender', 'race', 'arrival_transport']
train[categorical_cols] = train[categorical_cols].astype('category')

In [26]:
train['arrival_transport'].unique()

['AMBULANCE', 'WALK IN', 'UNKNOWN', 'OTHER', 'HELICOPTER']
Categories (5, object): ['AMBULANCE', 'HELICOPTER', 'OTHER', 'UNKNOWN', 'WALK IN']

In [ ]:
X = train[vital_signs_cols+categorical_cols+]

,subject_id,stay_id,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,chiefcomplaint,gender,anchor_age,anchor_year_group,race,arrival_transport,has_null
0,10000032,32952584,0.098914,0.369099,0.007235,0.096326,0.000463,0.000661,0.052239,2.0,Hypotension,F,52.0,2014 - 2016,WHITE,AMBULANCE,False
1,10000032,33258284,0.099523,0.296137,0.008348,0.096326,0.000695,0.000968,0.000000,3.0,"Abd pain, Abdominal distention",F,52.0,2014 - 2016,WHITE,AMBULANCE,False
2,10000032,35968195,0.100538,0.446352,0.009461,0.095333,0.000695,0.000876,0.074627,3.0,"n/v/d, Abd pain",F,52.0,2014 - 2016,WHITE,AMBULANCE,False
3,10000032,38112554,0.100030,0.373391,0.009461,0.096326,0.000761,0.001352,0.074627,3.0,Abdominal distention,F,52.0,2014 - 2016,WHITE,AMBULANCE,False
4,10000032,39399961,0.099828,0.326180,0.008348,0.097319,0.000629,0.000768,0.097015,2.0,"Abdominal distention, Abd pain, LETHAGIC",F,52.0,2014 - 2016,WHITE,AMBULANCE,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105863,19998562,36010073,0.099117,0.270386,0.009461,0.098312,0.000662,0.000860,0.000000,2.0,Syncope,M,79.0,2014 - 2016,WHITE,AMBULANCE,False
105864,19999043,33862917,0.099016,0.296137,0.008348,0.098312,0.000688,0.001152,0.067164,3.0,Vaginal bleeding,F,35.0,2014 - 2016,WHITE,AMBULANCE,False
105865,19999043,36370988,0.099117,0.356223,0.008348,0.099305,0.000721,0.001075,0.044776,3.0,Vaginal bleeding,F,35.0,2014 - 2016,WHITE,AMBULANCE,False
105866,19999043,39443857,0.099016,0.309013,0.009461,0.099305,0.000860,0.001352,0.037313,2.0,Vaginal bleeding,F,35.0,2014 - 2016,WHITE,WALK IN,False


In [36]:
# Process chief complaints in batches of 10,000
print("Computing symptom embeddings in batches...")
# Create embeddings for the chiefcomplaint column for train and test datasets
train_texts = train['chiefcomplaint'].tolist()
test_texts = test['chiefcomplaint'].tolist()

# Use the model to encode the texts; show_progress_bar=True gives you a progress update
train_embeddings = model.encode(train_texts, show_progress_bar=True)
test_embeddings = model.encode(test_texts, show_progress_bar=True)

# Save the embeddings as .npy files using NumPy
np.save('train_chiefcomplaint_embeddings.npy', train_embeddings)
np.save('test_chiefcomplaint_embeddings.npy', test_embeddings)

Computing symptom embeddings in batches...


Batches: 100%|██████████| 32/32 [00:01<00:00, 25.26it/s]


In [33]:
train_full = pd.read_csv('../data/mimic-iv-private/anchor_year_group_datasets/2014_-_2016/train_dataset.csv')
train_full['chiefcomplaint'] = train_full['chiefcomplaint'].astype(str)
train_full_texts = train_full['chiefcomplaint'].tolist()
train_full_embeddings = model.encode(train_full_texts, show_progress_bar=True)
np.save('train_full_chiefcomplaint_embeddings.npy', train_full_embeddings)


Batches: 100%|██████████| 3278/3278 [01:58<00:00, 27.56it/s]


In [ ]:

# Convert train symptom embeddings to a numpy array
train_full_embeddings = np.array(train_full_embeddings)

# Perform PCA on train symptom embeddings to reduce dimensionality to 10
print("Performing PCA on train symptom embeddings...")
pca = PCA(n_components=10)
train_full_embeddings_reduced = pca.fit_transform(train_full_embeddings)

# Save the reduced and comprehensive train embeddings to disk
np.save('../data/mimic-iv-private/train_full_embeddings_reduced.npy', train_full_embeddings_reduced)


Performing PCA on train symptom embeddings...


In [37]:
import numpy as np
from sklearn.decomposition import PCA

# ----- For the Training Set -----

# Convert train symptom embeddings to a numpy array
train_symptom_embeddings = np.array(train_embeddings)

# Perform PCA on train symptom embeddings to reduce dimensionality to 10
print("Performing PCA on train symptom embeddings...")
pca = PCA(n_components=10)
train_symptom_embeddings_reduced = pca.fit_transform(train_symptom_embeddings)

# Save the reduced and comprehensive train embeddings to disk
np.save('../data/mimic-iv-private/train_symptom_embeddings_reduced.npy', train_symptom_embeddings_reduced)


# ----- For the Test Set -----

# Convert test symptom embeddings to a numpy array
test_symptom_embeddings = np.array(test_embeddings)

# Use the same PCA transformation fitted on the training set to transform the test symptom embeddings
print("Performing PCA on test symptom embeddings...")
test_symptom_embeddings_reduced = pca.transform(test_symptom_embeddings)

# Save the reduced and comprehensive test embeddings to disk
np.save('../data/mimic-iv-private/test_symptom_embeddings_reduced.npy', test_symptom_embeddings_reduced)


Performing PCA on train symptom embeddings...
Performing PCA on test symptom embeddings...


In [64]:
vital_signs_normalized.shape
symptom_embeddings_reduced.shape

(100, 10)

In [55]:
embeddings = np.load('./data/mimic-iv-private/symptom_embeddings_batch_0.npy', allow_pickle=True)
similarities = model.similarity(embeddings, embeddings)


In [60]:
# Iterate through the first 100 rows
num_samples = 100
for i in range(num_samples):
    # Get the similarity scores for the current sample
    similarity_row = similarities[i]

    # Exclude self-similarity by setting its score to a very low value
    similarity_row[i] = -np.inf

    # Find the index of the most similar sample
    closest_index = np.argmax(similarity_row)

    # Retrieve and print the symptoms
    symptom = data['chiefcomplaint'].loc[i]
    closest_symptom = data['chiefcomplaint'].loc[closest_index.item()]

    print(f"Sample {i}: {symptom}")
    print(f"Most similar symptom: {closest_symptom}")
    print()

Sample 0: R Ankle pain
Most similar symptom: R Flank pain

Sample 1: Headache
Most similar symptom: Headache

Sample 2: Diarrhea, Presyncope
Most similar symptom: Diarrhea, Weakness

Sample 3: RLQ abdominal pain
Most similar symptom: RLQ ABDOMINAL PAIN

Sample 4: SOB
Most similar symptom: SI

Sample 5: Agitation, SI
Most similar symptom: SI

Sample 6: Headache
Most similar symptom: Headache

Sample 7: Back pain
Most similar symptom: BACK PAIN

Sample 8: Buttock pain, L Leg pain
Most similar symptom: Buttock pain, Pelvic pain

Sample 9: R Hand injury
Most similar symptom: R Hand injury, Syncope

Sample 10: KIDNEY STONES
Most similar symptom: Dysuria

Sample 11: Fever, Cough
Most similar symptom: Dyspnea, Fever

Sample 12: N/V, RENAL FAILURE
Most similar symptom: N/V

Sample 13: Abd pain
Most similar symptom: ABD PAIN

Sample 14: Laceration, Altered mental status
Most similar symptom: Altered mental status

Sample 15: Chest pain, Dyspnea
Most similar symptom: Chest pain, Dyspnea

Sample 

In [ ]:

# Save the final combined embeddings to CSV
print("Saving final comprehensive embeddings to CSV...")
comprehensive_embeddings_df = pd.DataFrame(comprehensive_embeddings)
comprehensive_embeddings_df.to_csv('./data/mimic-iv-private/comprehensive_embeddings.csv', index=False)

print("Process completed successfully!")

## Compute Embeddings

In [ ]:
symptom_embeddings = model.encode(
    data['chiefcomplaint'][0],
    batch_size=1,
    convert_to_numpy=True
)

In [4]:
# Compute embeddings for 'chiefcomplaint'
print("Computing symptom embeddings...")
symptom_embeddings = model.encode(
    test_data['chiefcomplaint'].tolist(),
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True
)


Computing symptom embeddings...


Batches:   0%|          | 0/40 [00:00<?, ?it/s]

Batches: 100%|██████████| 40/40 [00:04<00:00,  8.08it/s]


In [6]:
symptom_embeddings.shape

(2500, 768)

In [7]:
np.save('./data/mimic-iv-private/symptom_embeddings_test.npy', symptom_embeddings)


In [26]:
# Perform PCA on symptom embeddings to reduce dimensionality to 30
print("Performing PCA on symptom embeddings...")
pca = PCA(n_components=14)
symptom_embeddings_reduced = pca.fit_transform(symptom_embeddings)

Performing PCA on symptom embeddings...


In [ ]:
# embeddings = np.load('./data/mimic-iv-private/comprehensive_embeddings.npy',allow_pickle=True)

In [63]:
import numpy as np
import pandas as pd
from collections import Counter


def compute_cosine_similarity(embedding, embeddings, epsilon=1e-8):
    """
    Compute cosine similarity between a single embedding and a set of embeddings.
    Avoids division by zero by adding epsilon to the norm.
    """
    # Compute cosine similarities
    similarities = np.dot(embeddings, embedding)
    return similarities

def get_top_k_similar(embedding, embeddings, k=5):
    """
    Find the top-k most similar samples to a given embedding.
    """
    similarities = compute_cosine_similarity(embedding, embeddings)
    top_k_indices = np.argsort(similarities)[-k:][::-1]
    return top_k_indices, similarities[top_k_indices], similarities

patient = 8
# Get top 5 similar embeddings
top_k_indices, top_k_similarities, similarities = get_top_k_similar(symptom_embeddings[patient], symptom_embeddings, k=25)

# Retrieve and print the corresponding symptoms
print(f"Querying {data['chiefcomplaint'].iloc[patient]} most similar samples:")
for i, index in enumerate(top_k_indices):
    symptom = data['chiefcomplaint'].iloc[index]
    acuity = data['acuity'].iloc[index]
    similarity = top_k_similarities[i]
    print(f"Rank {i+1}: {symptom} | Index: {index} | Similarity: {similarity:.4f} | Acuity: {acuity}")


Querying Buttock pain, L Leg pain most similar samples:
Rank 1: Buttock pain, L Leg pain | Index: 8 | Similarity: 182.3298 | Acuity: 3.0
Rank 2: Buttock pain, L Leg pain | Index: 123029 | Similarity: 182.3297 | Acuity: 3.0
Rank 3: BUTTOCK, LEG PAIN | Index: 235208 | Similarity: 175.0675 | Acuity: 3.0
Rank 4: R BUTTOCK/LEG PAIN | Index: 246717 | Similarity: 172.5656 | Acuity: 3.0
Rank 5: Buttock pain, Leg pain | Index: 302301 | Similarity: 172.5599 | Acuity: 4.0
Rank 6: Leg pain, L Leg pain | Index: 364963 | Similarity: 170.6013 | Acuity: 3.0
Rank 7: Body pain, L Leg pain | Index: 114039 | Similarity: 169.9877 | Acuity: 3.0
Rank 8: Buttock pain, R Leg pain | Index: 282507 | Similarity: 169.9574 | Acuity: 3.0
Rank 9: Buttock pain, R Leg pain | Index: 226989 | Similarity: 169.9574 | Acuity: 3.0
Rank 10: L BUTTOCK PAIN | Index: 345878 | Similarity: 168.6251 | Acuity: 1.0
Rank 11: L BUTTOCK PAIN | Index: 100046 | Similarity: 168.6251 | Acuity: 4.0
Rank 12: L BUTTOCK PAIN | Index: 331936 | S

In [64]:
top_k_indices

array([     8, 123029, 235208, 246717, 302301, 364963, 114039, 282507,
       226989, 345878, 100046, 331936,  46441, 189041, 273005, 171619,
        29373, 163178, 248571, 111618, 166571, 322698, 311573, 165287,
       230436])

Now, let's add the vitals and repeat the analysis -- let's first try cosine

In [65]:

# Example vital sign columns in your dataframe
vital_sign_columns = ['temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp', 'pain']

# Extract the query sample's vital signs
query_vital_signs = data.loc[top_k_indices[0], vital_sign_columns].values.astype(float)

# Extract the vital signs for the top 25 samples
top_25_vital_signs = data.loc[top_k_indices, vital_sign_columns].values.astype(float)

# Normalize the query and top 25 vital signs
query_vital_signs_normalized = query_vital_signs / np.linalg.norm(query_vital_signs)
top_25_vital_signs_normalized = top_25_vital_signs / np.linalg.norm(top_25_vital_signs, axis=1, keepdims=True)

# Compute cosine similarity
vital_signs_similarity = np.dot(top_25_vital_signs_normalized, query_vital_signs_normalized)

# Get the index of the most similar sample based on vital signs
most_similar_index = np.argmax(vital_signs_similarity)
most_similar_sample = data.loc[top_k_indices[most_similar_index]]

# Retrieve the top-k similar samples based on vital signs
top_k_vital_indices = np.argsort(vital_signs_similarity)[-5:][::-1]
top_k_vital_samples = data.loc[top_k_indices[top_k_vital_indices]]

# Output results
print(f"Most similar sample (based on vital signs):")
print(most_similar_sample)

print("\nTop 5 most similar samples (based on vital signs):")
print(top_k_vital_samples)

Most similar sample (based on vital signs):
subject_id                        16052047
stay_id                           36556660
temperature                           98.5
heartrate                             58.0
resprate                              14.0
o2sat                                100.0
sbp                                  151.0
dbp                                   70.0
pain                                   6.0
acuity                                 3.0
chiefcomplaint    Buttock pain, L Leg pain
Name: 8, dtype: object

Top 5 most similar samples (based on vital signs):
        subject_id   stay_id  temperature  heartrate  resprate  o2sat    sbp  \
8         16052047  36556660         98.5       58.0      14.0  100.0  151.0   
246717    17010538  31731207         98.4       60.0      18.0  100.0  164.0   
163178    16531643  36827756         98.4       63.0      19.0  100.0  142.0   
235208    17113027  31142049         98.3       71.0      16.0  100.0  138.0   
165287  

In [55]:
# Example vital sign columns in your dataframe
vital_sign_columns = ['temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp', 'pain']

# Extract the query sample's vital signs
query_vital_signs = data.loc[top_k_indices[0], vital_sign_columns].values.astype(float)

# Extract the vital signs for the top 25 samples
top_25_vital_signs = data.loc[top_k_indices, vital_sign_columns].values.astype(float)

# Compute Euclidean distance between the query and the top 25 samples
distances = np.linalg.norm(top_25_vital_signs - query_vital_signs, axis=1)

# Get the index of the most similar sample based on Euclidean distance
most_similar_index = np.argmin(distances)
most_similar_sample = data.loc[top_k_indices[most_similar_index]]

# Retrieve the top-k similar samples based on Euclidean distance
top_k_vital_indices = np.argsort(distances)[:5]
top_k_vital_samples = data.loc[top_k_indices[top_k_vital_indices]]

# Output results
print(f"Most similar sample (based on Euclidean distance):")
print(most_similar_sample)

print("\nTop 5 most similar samples (based on Euclidean distance):")
print(top_k_vital_samples)

Most similar sample (based on Euclidean distance):
subject_id            15289989
stay_id               36797979
temperature               97.9
heartrate                 92.0
resprate                  20.0
o2sat                    100.0
sbp                      143.0
dbp                       82.0
pain                      10.0
acuity                     3.0
chiefcomplaint    R Ankle pain
Name: 0, dtype: object

Top 5 most similar samples (based on Euclidean distance):
        subject_id   stay_id  temperature  heartrate  resprate  o2sat    sbp  \
0         15289989  36797979         97.9       92.0      20.0  100.0  143.0   
62199     12618057  37799336         98.6       91.0      16.0  100.0  148.0   
224967    13524705  32250368         97.8       85.0      18.0   98.0  150.0   
222528    12672071  31541890         96.6      103.0      18.0   99.0  146.0   
222646    19456368  35401303         98.4       80.0      20.0  100.0  146.0   

         dbp  pain  acuity chiefcomplaint  
0

## Run more scripts

In [ ]:
!python llm-predict-triage.py --dataset Triage-Private-Stratified --start 0 --end 0 --model openai-gpt-4o-chat --strategy KATE --json --debug --k_shots 10

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading Dataset...
Potentially loading existing predictions...
Making 1 new predictions...
0it [00:00, ?it/s]Estimate the patient's acuity from 1-5 based on the following guidelines: Acuity is assessed using the Emergency Severity Index (ESI) Five Level triage system. This priority is assigned by a registered nurse. Level 1 is the highest priority, while level 5 is the lowest priority. Answer in valid JSON format, providing acuity as a single numeric value in the key 'acuity'.

Input: temperature   heartrate   resprate   o2sat   sbp   dbp   pain   chiefcomplaint
99.3   92.0   24.0   96.0   134.0   57.0   0   Dyspnea on exertion
Output: 2.0

Input: temperature   heartrate   resprate   o2sat   sbp   dbp   pain   chiefcomplaint
98.3   111.0   16.0   100.0   125.0   60.0   0   Dyspnea on exertion
Output: 2.0

Input: temperature   heartrate   resprate   o2sat   sbp   dbp   pain   chiefcomplaint
98.2   72.0   18.0   100.0   134.0   72.0   0   Dyspnea on exertion
Output: 2.0

Input: temperatu

In [84]:
!python llm-evaluate-triage.py --dataset Triage-Private-Stratified --start 0 --end 2499 --ordinal --parameters Triage-Private-Stratified_FewShot_openai-gpt-4o-chat_json0_2500_20241127_230336 --by_class

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading Dataset...
2500
Loading Predictions...
Overall Metrics: {'overall': {'accuracy': 0.566, 'precision': 0.6377718494209463, 'recall': 0.566, 'f1_score': 0.5786089315616268, 'adjusted_accuracy': 0.9564, 'adjusted_precision': 0.9679252936610608, 'adjusted_recall': 0.9564, 'adjusted_f1': 0.9593395733899968, 'mae': 0.4796, 'mse': 0.5748, 'quadratic_kappa': 0.4725431743783277}, 'by_class': {'1.0': {'precision': 0.6666666666666666, 'recall': 0.046511627906976744, 'f1-score': 0.08695652173913043, 'support': 86.0}, '2.0': {'precision': 0.5937149270482603, 'recall': 0.6320191158900836, 'f1-score': 0.6122685185185185, 'support': 837.0}, '3.0': {'precision': 0.7173708920187793, 'recall': 0.548456568557071, 'f1-score': 0.6216436126932465, 'support': 1393.0}, '4.0': {'precision': 0.2261904761904762, 'recall': 0.6440677966101694, 'f1-score': 0.33480176211453744, 'support': 177.0}, '5.0': {'precision': 0.11764705882352941, 'recall': 0.5714285714285714, 'f1-score': 0.1951219512195122, 'support': 

In [85]:
df = pd.read_csv('./results/Triage-Private-Stratified/Triage-Private-Stratified_ZeroShot_openai-gpt-4o-chat_json_detailed0_1000_20241125_170724.csv')

In [93]:
df

,prompt,Estimated_Acuity,Reasoning,subject_id,stay_id,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,chiefcomplaint
0,Here is the profile of a patient:\n\ntemperatu...,3,NaN,18474069,30615360,98.2,71.0,18.0,94.0,92.0,36.0,0,3.0,Dyspnea on exertion
1,NaN,4,NaN,10482402,30835613,97.3,68.0,18.0,100.0,131.0,74.0,5,3.0,S/P FALL
2,NaN,2,NaN,11668089,30163418,97.6,105.0,22.0,100.0,147.0,76.0,3,2.0,"Chest pain, Dyspnea"
3,NaN,2,NaN,17170624,35921297,97.6,110.0,16.0,98.0,99.0,65.0,10,3.0,Abnormal CT
4,NaN,5,NaN,17532289,37034357,97.7,85.0,20.0,100.0,134.0,76.0,0,4.0,Rash
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,NaN,2,NaN,12003826,33426441,98.8,98.0,18.0,98.0,159.0,91.0,10,3.0,Back pain
696,NaN,2,NaN,17125290,33272361,97.9,100.0,18.0,99.0,105.0,83.0,8,2.0,Inguinal pain
697,NaN,3,NaN,15964158,32546263,98.6,75.0,12.0,98.0,127.0,86.0,unable,3.0,"ETOH, s/p Fall"
698,NaN,2,NaN,13528930,37977947,98.0,120.0,16.0,100.0,156.0,66.0,0,3.0,"WEAKNESS, FATIGUE"


In [88]:
filename ='Triage-Private-Stratified_ZeroShot_openai-gpt-4o-chat_json_detailed600_1000_20241126_024228.csv'
df2 = pd.read_csv(f'./results/Triage-Private-Stratified/{filename}')
df2

,Estimated_Acuity,Reasoning,subject_id,stay_id,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,chiefcomplaint
0,2,NaN,18159815,31608693,99.7,90.0,18.0,98.0,118.0,77.0,0,2.0,"Allergic reaction, Dyspnea"
1,2,NaN,14421355,32098882,99.4,132.0,16.0,100.0,120.0,72.0,7,1.0,"Headache, Sore throat"
2,4,NaN,18505007,36748085,98.6,96.0,18.0,100.0,137.0,90.0,0,4.0,"Dysuria, Urinary frequency"
3,3,NaN,17113137,38284564,98.9,102.0,22.0,96.0,125.0,75.0,0,2.0,___
4,3,NaN,15675253,30288651,98.1,72.0,16.0,100.0,127.0,56.0,5,3.0,"Back pain, MVC"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,2,NaN,17089649,38944330,98.8,114.0,15.0,100.0,148.0,102.0,10,4.0,L Hip pain
397,3,NaN,19343822,39796457,97.2,82.0,16.0,97.0,129.0,57.0,4,3.0,"Abd pain, Chest pain, N/V"
398,3,NaN,12352918,33269944,98.7,106.0,16.0,100.0,136.0,76.0,5,3.0,"MVC, Neck pain"
399,4,NaN,13673417,35894831,99.5,73.0,16.0,97.0,124.0,53.0,3,3.0,LEFT ANKLE PAIN


In [90]:
filename ='Triage-Private-Stratified_ZeroShot_openai-gpt-4o-chat_json_detailed1000_2500_20241127_160229.csv'
df3 = pd.read_csv(f'./results/Triage-Private-Stratified/{filename}')
df3

,Estimated_Acuity,Reasoning,subject_id,stay_id,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,chiefcomplaint
0,3,NaN,19481752,35968442,98.7,114.0,18.0,98.0,130.0,82.0,0,3.0,"Epigastric pain, N/V"
1,5,NaN,17148408,36757338,98.5,58.0,18.0,100.0,104.0,48.0,0,2.0,DIZZINESS
2,2,NaN,18676703,32629111,97.6,103.0,22.0,100.0,237.0,119.0,10,2.0,"N/V, Hyperglycemia"
3,2,NaN,13247319,30478070,98.3,82.0,18.0,99.0,147.0,92.0,0,3.0,CHEST FLUTTERING
4,3,NaN,12782393,30518271,99.2,86.0,17.0,98.0,119.0,70.0,3,3.0,Abd pain
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,4,NaN,12791002,30386504,98.2,63.0,18.0,100.0,149.0,64.0,2,3.0,WOUND EVAL
1496,2,NaN,12938336,34253493,98.2,88.0,18.0,97.0,143.0,63.0,0,2.0,Hemoptysis
1497,2,NaN,13378145,35205258,98.0,114.0,16.0,99.0,131.0,93.0,0,2.0,Seizure
1498,2,NaN,12351481,39111882,100.0,81.0,20.0,92.0,130.0,56.0,0,2.0,Dyspnea


In [94]:
combined_df = pd.concat([df.loc[:599], df2.loc[:399], df3], ignore_index=True)
combined_df.to_csv('Triage-Private-Stratified_ZeroShot_openai-gpt-4o-chat_json_detailed0_2500_20241127.csv'
)

In [95]:
!python llm-evaluate-triage.py --dataset Triage-Private-Stratified --start 0 --end 2499 --ordinal --parameters Triage-Private-Stratified_ZeroShot_openai-gpt-4o-chat_json_detailed0_2500_20241127

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading Dataset...
2500
Loading Predictions...
Overall Metrics: {'overall': {'accuracy': 0.4172, 'precision': 0.5540308291028618, 'recall': 0.4172, 'f1_score': 0.4214614602283862, 'adjusted_accuracy': 0.882, 'adjusted_precision': 0.9471513342295009, 'adjusted_recall': 0.882, 'adjusted_f1': 0.9119957692487527, 'mae': 0.7208, 'mse': 1.04, 'quadratic_kappa': 0.2621665333269161}}
Evaluation complete. Metrics and plots saved.


In [96]:
!python llm-evaluate-triage.py --dataset Triage-Private-Stratified --start 0 --end 2499 --ordinal --parameters Triage-Private-Stratified_ZeroShot_openai-gpt-4o-chat_json0_2500_20241127_084108

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading Dataset...
2500
Loading Predictions...
Overall Metrics: {'overall': {'accuracy': 0.5532, 'precision': 0.6364463639662771, 'recall': 0.5532, 'f1_score': 0.5643732115298273, 'adjusted_accuracy': 0.922, 'adjusted_precision': 0.9633680383607801, 'adjusted_recall': 0.922, 'adjusted_f1': 0.9373247965516198, 'mae': 0.536, 'mse': 0.7368, 'quadratic_kappa': 0.3668244037012104}}
Evaluation complete. Metrics and plots saved.


### Bias Scripts

In [98]:
!python llm-predict-triage.py --dataset Triage-Counterfactual --start 0 --end 3000 --model gpt-4o-mini --strategy ZeroShot --bias --json --vitals_off

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading Dataset...
Potentially loading existing predictions...
Making 3001 new predictions...
99it [01:08,  1.20it/s]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_ZeroShot_gpt-4o-mini_json0_100_20241128_020646.csv
Saved progress after 100 predictions.
199it [02:17,  1.33s/it]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_ZeroShot_gpt-4o-mini_json0_200_20241128_020756.csv
Saved progress after 200 predictions.
299it [03:24,  1.18it/s]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_ZeroShot_gpt-4o-mini_json0_300_20241128_020903.csv
Saved progress after 300 predictions.
399it [04:42,  1.44it/s]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_ZeroShot_gpt-4o-mini_json0_400_20241128_021020.csv
Saved progress after 400 predictions.
499it [05:53,  1.49it/s]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_ZeroShot_gpt-4o-mini_json0_500_20241128_021131.csv
Saved progress 

In [99]:
!python llm-predict-triage.py --dataset Triage-Counterfactual --start 0 --end 3000 --model gpt-3.5-turbo --strategy ZeroShot --bias --json --vitals_off

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading Dataset...
Potentially loading existing predictions...
Making 3001 new predictions...
99it [00:51,  1.93it/s]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_ZeroShot_gpt-3.5-turbo_json0_100_20241128_023523.csv
Saved progress after 100 predictions.
199it [01:43,  1.97it/s]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_ZeroShot_gpt-3.5-turbo_json0_200_20241128_023615.csv
Saved progress after 200 predictions.
299it [02:43,  1.16it/s]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_ZeroShot_gpt-3.5-turbo_json0_300_20241128_023715.csv
Saved progress after 300 predictions.
399it [03:37,  1.94it/s]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_ZeroShot_gpt-3.5-turbo_json0_400_20241128_023809.csv
Saved progress after 400 predictions.
499it [04:34,  1.54it/s]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_ZeroShot_gpt-3.5-turbo_json0_500_20241128_023906.csv
Saved

In [100]:
!python llm-predict-triage.py --dataset Triage-Counterfactual --start 0 --end 3000 --model gpt-4o --strategy ZeroShot --bias --json --vitals_off

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading Dataset...
Potentially loading existing predictions...
Making 3001 new predictions...
99it [01:39,  1.38it/s]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_ZeroShot_gpt-4o_json0_100_20241128_030730.csv
Saved progress after 100 predictions.
199it [03:18,  1.06s/it]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_ZeroShot_gpt-4o_json0_200_20241128_030910.csv
Saved progress after 200 predictions.
299it [04:49,  1.60it/s]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_ZeroShot_gpt-4o_json0_300_20241128_031040.csv
Saved progress after 300 predictions.
399it [06:25,  1.10it/s]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_ZeroShot_gpt-4o_json0_400_20241128_031217.csv
Saved progress after 400 predictions.
499it [08:20,  1.17it/s]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_ZeroShot_gpt-4o_json0_500_20241128_031411.csv
Saved progress after 500 predictions.
59

In [101]:
!python llm-predict-triage.py --dataset Triage-Counterfactual --start 0 --end 3000 --model claude-3-haiku-20240307 --strategy ZeroShot --bias --json --vitals_off

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading Dataset...
Potentially loading existing predictions...
Making 3001 new predictions...
99it [00:44,  2.46it/s]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_ZeroShot_claude-3-haiku-20240307_json0_100_20241128_033957.csv
Saved progress after 100 predictions.
199it [01:38,  2.18it/s]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_ZeroShot_claude-3-haiku-20240307_json0_200_20241128_034051.csv
Saved progress after 200 predictions.
299it [02:27,  2.50it/s]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_ZeroShot_claude-3-haiku-20240307_json0_300_20241128_034140.csv
Saved progress after 300 predictions.
399it [03:20,  1.94it/s]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_ZeroShot_claude-3-haiku-20240307_json0_400_20241128_034233.csv
Saved progress after 400 predictions.
499it [04:12,  2.81it/s]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_ZeroShot_claude-3-

In [102]:
!python llm-predict-triage.py --dataset Triage-Counterfactual --start 0 --end 3000 --model claude-3-sonnet-20240229 --strategy ZeroShot --bias --json --vitals_off

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading Dataset...
Potentially loading existing predictions...
Making 3001 new predictions...
99it [01:15,  1.77it/s]Error calling Claude: Error code: 500 - {'type': 'error', 'error': {'type': 'api_error', 'message': 'Internal server error'}}
Error making API call: expected string or bytes-like object
DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_ZeroShot_claude-3-sonnet-20240229_json0_100_20241128_042203.csv
Saved progress after 100 predictions.
199it [02:53,  1.15s/it]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_ZeroShot_claude-3-sonnet-20240229_json0_200_20241128_042309.csv
Saved progress after 200 predictions.
231it [03:13,  1.75it/s]Error calling Claude: Error code: 500 - {'type': 'error', 'error': {'type': 'api_error', 'message': 'Internal server error'}}
Error making API call: expected string or bytes-like object
299it [04:33,  1.14it/s]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_ZeroShot_claude-

In [103]:
!python llm-predict-triage.py --dataset Triage-Counterfactual --start 0 --end 3000 --model claude-3-sonnet-20240229 --strategy CoT --bias --json --vitals_off

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading Dataset...
Potentially loading existing predictions...
Making 3001 new predictions...
99it [06:55,  3.11s/it]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_CoT_claude-3-sonnet-20240229_json0_100_20241128_045559.csv
Saved progress after 100 predictions.
199it [13:52,  3.69s/it]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_CoT_claude-3-sonnet-20240229_json0_200_20241128_050256.csv
Saved progress after 200 predictions.
299it [30:47,  3.86s/it] DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_CoT_claude-3-sonnet-20240229_json0_300_20241128_051953.csv
Saved progress after 300 predictions.
399it [37:39,  4.24s/it]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_CoT_claude-3-sonnet-20240229_json0_400_20241128_052645.csv
Saved progress after 400 predictions.
430it [39:56,  4.38s/it]Error calling Claude: Error code: 500 - {'type': 'error', 'error': {'type': 'api_error', 'message': 'Inter

In [104]:
!python llm-predict-triage.py --dataset Triage-Counterfactual --start 0 --end 3000 --model claude-3-haiku-20240307 --strategy CoT --bias --json --vitals_off

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading Dataset...
Potentially loading existing predictions...
Making 3001 new predictions...
99it [02:59,  1.98s/it]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_CoT_claude-3-haiku-20240307_json0_100_20241128_075414.csv
Saved progress after 100 predictions.
199it [06:11,  1.74s/it]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_CoT_claude-3-haiku-20240307_json0_200_20241128_075726.csv
Saved progress after 200 predictions.
299it [09:17,  2.12s/it]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_CoT_claude-3-haiku-20240307_json0_300_20241128_080032.csv
Saved progress after 300 predictions.
332it [10:20,  1.87s/it]Error decoding JSON: Expecting ',' delimiter: line 1 column 830 (char 829)
Raw response causing error: {  "reasoning": "Based on the given information, the patient's chief complaint is 'Wound eval, Transfer', which suggests a potential medical issue that requires evaluation and possible transfer to anot

In [105]:
!python llm-predict-triage.py --dataset Triage-Counterfactual --start 0 --end 3000 --model gpt-3.5-turbo --strategy CoT --bias --json --vitals_off

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading Dataset...
Potentially loading existing predictions...
Making 3001 new predictions...
99it [02:07,  1.32s/it]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_CoT_gpt-3.5-turbo_json0_100_20241128_090705.csv
Saved progress after 100 predictions.
199it [04:21,  1.34s/it]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_CoT_gpt-3.5-turbo_json0_200_20241128_090918.csv
Saved progress after 200 predictions.
299it [06:37,  1.32s/it]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_CoT_gpt-3.5-turbo_json0_300_20241128_091135.csv
Saved progress after 300 predictions.
399it [08:43,  1.28s/it]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_CoT_gpt-3.5-turbo_json0_400_20241128_091341.csv
Saved progress after 400 predictions.
499it [10:48,  1.31s/it]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_CoT_gpt-3.5-turbo_json0_500_20241128_091546.csv
Saved progress after 500 predi

In [106]:
!python llm-predict-triage.py --dataset Triage-Counterfactual --start 0 --end 3000 --model gpt-4o-mini --strategy CoT --bias --json --vitals_off

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading Dataset...
Potentially loading existing predictions...
Making 3001 new predictions...
99it [04:57,  2.44s/it]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_CoT_gpt-4o-mini_json0_100_20241128_101114.csv
Saved progress after 100 predictions.
199it [10:34,  3.33s/it]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_CoT_gpt-4o-mini_json0_200_20241128_101651.csv
Saved progress after 200 predictions.
299it [15:39,  3.25s/it]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_CoT_gpt-4o-mini_json0_300_20241128_102157.csv
Saved progress after 300 predictions.
399it [20:23,  3.53s/it]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_CoT_gpt-4o-mini_json0_400_20241128_102641.csv
Saved progress after 400 predictions.
499it [25:11,  2.89s/it]DataFrame saved to ./results/Triage-Counterfactual/Triage-Counterfactual_CoT_gpt-4o-mini_json0_500_20241128_103128.csv
Saved progress after 500 predictions.
59